In [13]:
import youtube_dl
import os
import sys
import shutil
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
from subprocess import Popen, PIPE
import pandas as pd

import utils

from cvcalib.audiosync import offset

curr_dir = "/home/sean/Documents/experiments/stage-mix-generator/"
download_dir = os.path.join(curr_dir, 'temp')

In [2]:
song_text = "Boy With Luv BTS"
lyrics_video_url = ""
stage_video_urls = []

In [3]:
textToSearch = song_text+' lyrics'

query = urllib.parse.quote(textToSearch)
url = "https://www.youtube.com/results?search_query=" + query
response = urllib.request.urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'}):
    if 'stage mix' not in vid['title'].lower():
        lyrics_video_url = 'https://www.youtube.com' + vid['href']
        break

In [4]:
textToSearch = song_text+' stage live'

query = urllib.parse.quote(textToSearch)
url = "https://www.youtube.com/results?search_query=" + query
response = urllib.request.urlopen(url)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')
for vid in soup.findAll(attrs={'class':'yt-uix-tile-link'}):
    if 'stage mix' not in vid['title'].lower():
        stage_video_urls.append('https://www.youtube.com' + vid['href'])
        
        if len(stage_video_urls) >= 5:
            break

In [5]:
# download videos

ydl_opts = {
    'format': 'mp4',
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    
    meta = ydl.extract_info(
        lyrics_video_url, download=False)
    
    print(meta['formats'])
    
    lyrics_audio = None
    for fmt in meta['formats']:
        if fmt['format_id'] == 140:
            
    
    ydl.download([yrics_video_url])
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, "lyrics_video.mp4"))
    
    for i, url in enumerate(stage_video_urls):
        ydl.download([url])
        meta = ydl.extract_info(
            url, download=False)
        
        shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, f"stage_video{i}.mp4"))

IndentationError: expected an indented block (<ipython-input-5-9fe99a192f0b>, line 19)

In [ ]:
ydl_opts = {
    'format': 'm4a', # choice of quality
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    meta = ydl.extract_info('https://www.youtube.com/watch?v=zzq8uI1BnJk', download=False)
    ydl.download(['https://www.youtube.com/watch?v=zzq8uI1BnJk'])
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.m4a"), os.path.join(download_dir, "lyrics_video.m4a"))

In [6]:
lyrics_video_path = os.path.join(download_dir, 'lyrics_video.mp4')
stage_video_paths = [ os.path.join(download_dir, f'stage_video{i}.mp4') for i in range(len(stage_video_urls)) ]

# find timestamp of start of song in stage videos
offset.find_time_offset(['lyrics_video.mp4', 'stage_video0.mp4'], download_dir+'/', [0, 0])

RuntimeError: ffmpeg error:
b'ffmpeg version 4.1.3 Copyright (c) 2000-2019 the FFmpeg developers\r\n  built with gcc 8.3.1 (GCC) 20190414\r\n  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth\r\n  libavutil

In [ ]:
import os.path as osp
from subprocess import Popen, PIPE
folder = download_dir
audio_file = 'lyrics_audio.m4a'
track_name = audio_file.split(".")
audio_output_name = track_name[0] + "WAV.wav"  # !! CHECK TO SEE IF FILE IS IN UPLOADS DIRECTORY
audio_output_path = osp.join(folder,audio_output_name)
#ffmpeg was accepted back into Debian, and libav is now a 2nd choice, so, what gives? -Greg
#Now piping the output back to this process (to reduce verbosity and potentially be able to use
#the output).-Greg
#TODO: detect whether user has avconv or ffmpeg, and use the appropriate call
args = ["ffmpeg", "-y", "-i", osp.join(folder, audio_file), "-ac", "1", "-f", "wav", audio_output_path]
process = Popen(args, stdout=PIPE, stderr=PIPE)
output, err = process.communicate()
exit_code = process.wait()
exit_code#, str(err), audio_output_path, str(output)

In [ ]:
stage_song_starts = []
for i in range(len(stage_video_urls)):
    output = offset.find_time_offset(['lyrics_video.mp4', f'stage_video{i}.mp4'], download_dir+'/', [0, 0])
    print(output)
    stage_song_starts.append(output[0])

In [15]:
DWNLD_DIR = os.path.join('.', 'temp')

song_audio_filepath = os.path.join(DWNLD_DIR, 'song_audio.m4a')
stage_video_filepaths = [
    os.path.join(DWNLD_DIR, 'stage_video0.mp4'), 
    os.path.join(DWNLD_DIR, 'stage_video1.mp4'), 
    os.path.join(DWNLD_DIR, 'stage_video2.mp4')
]
stage_audio_filepaths = [
    os.path.join(DWNLD_DIR, 'stage_audio0.m4a'), 
    os.path.join(DWNLD_DIR, 'stage_audio1.m4a'), 
    os.path.join(DWNLD_DIR, 'stage_audio2.m4a')
]

song_audio_filename = song_audio_filepath.split(os.path.sep)[-1]
stage_audio_filenames = [path.split(os.path.sep)[-1] for path in stage_audio_filepaths]

# find timestamp of start of song in stage videos
stage_song_starts = []
for i in range(len(stage_video_filepaths)):
    try:
        output = offset.find_time_offset([song_audio_filename, stage_audio_filenames[i]], DWNLD_DIR+os.path.sep, [0, 0])
        print(output)
        stage_song_starts.append(output[0])
    except Exception as e:
        print(e)
        stage_song_starts.append((1, 0))

# get subclips of the song in the stage videos
stage_videos_used, stage_videos, stage_audios = [], [], []
for i in range(len(stage_video_filepaths)):
    if stage_song_starts[i][0] == 0:
        stage_videos_used.append(i)
        stage_videos.append(VideoFileClip(stage_video_filepaths[i]).subclip(stage_song_starts[i][1]))
        stage_audios.append(AudioFileClip(stage_audio_filepaths[i]).subclip(stage_song_starts[i][1]))

if len(stage_videos) == 0:
    print("No suitible videos found")

# load song audio
song_audio = AudioFileClip(song_audio_filepath)

Could not determine frame rate from ffmpeg output. Printing output...
Could not determine frame rate from ffmpeg output. Printing output...
((54.4856, 0), 0.0)
Could not determine frame rate from ffmpeg output. Printing output...
Could not determine frame rate from ffmpeg output. Printing output...
((54.5321, 0), 0.0)
Could not determine frame rate from ffmpeg output. Printing output...
Could not determine frame rate from ffmpeg output. Printing output...
((50.8053, 0), 0.0)
No suitible videos found


In [ ]:
# get subclips of the song in the stage videos

# get length of lyrics video
lyrics_video = VideoFileClip(lyrics_video_path)

stage_videos = []
for i, path in enumerate(stage_video_paths):
    if stage_song_starts[i][0] == 0:
        stage_videos.append(VideoFileClip(path).subclip(stage_song_starts[i][1]))

In [ ]:
# every 5 seconds during the song, splice clips from different performances
clips = []
stage_video_idx = 0
for step in np.arange(0, lyrics_video.duration, 5):
    if step+5 < stage_videos[stage_video_idx].duration:
        clips.append(stage_videos[stage_video_idx].subclip(step, step+5))
    
    stage_video_idx = (stage_video_idx + 1) % len(stage_videos)

In [ ]:
clips = []
# find cuts in videos
video_scenes = []
for idx in stage_videos_used:
    video_scenes.append(utils.detect_scenes(stage_video_filepaths[idx]))

# assemble clips
curr_time = 0.0
while curr_time > float(song_audio.duration-10):
    print("curr_time:", curr_time)
    # choose random video
    rand_idx = randbelow(len(stage_videos))

    # skip shorter videos
    if curr_time > float(stage_videos[rand_idx].duration)+2:
        continue

    # get next cut in this video
    for i, scene in enumerate(video_scenes[rand_idx]):
        scene_end = utils.timecode_to_seconds(scene[1].get_timecode()) - stage_song_starts[rand_idx][1]
        if scene_end > curr_time+1.0 and scene_end < float(song_audio.duration):
            clips.append(stage_videos[rand_idx].subclip(curr_time, scene_end))   
            curr_time = scene_end

In [ ]:
final_clip = concatenate_videoclips(clips)

#final_clip = final_clip.set_duration(lyrics_video.duration)
#final_clip = final_clip.set_audio(lyrics_video.audio)

final_clip = final_clip.set_audio(stage_videos[0].audio)
final_clip.write_videofile("stage_mix.mp4")

In [ ]:
title = f"ITZY - DALLA DALLA - Stage Mix"
desc = "Videos Used: \n" + "\n".join([ 'dank' ])
args = [
    #f"PYTHONPATH={os.path.join(curr_dir, 'youtube-upload')}", 
    "python", 
    f"{os.path.join(curr_dir, 'youtube-upload', 'bin', 'youtube-upload')}",
    f"--title={title}",
    f"--description={desc}",
    f"{os.path.join(curr_dir, 'stage_mix.mp4')}",
    "--privacy",
    "private"
]
process = Popen(args, stdout=PIPE, stderr=PIPE)
output, err = process.communicate()
exit_code = process.wait()
print(exit_code, str(err), str(output))

In [ ]:
import scenedetect
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.frame_timecode import FrameTimecode
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.detectors import ThresholdDetector

import os

In [ ]:
video_manager = VideoManager(['stage_mix.mp4'])
stats_manager = StatsManager()
scene_manager = SceneManager(stats_manager)
# Add ContentDetector algorithm (constructor takes detector options like threshold).
scene_manager.add_detector(ContentDetector(threshold=40))
base_timecode = video_manager.get_base_timecode()

STATS_FILE_PATH = 'testvideo.stats.csv'

try:
    # If stats file exists, load it.
    if os.path.exists(STATS_FILE_PATH):
        # Read stats from CSV file opened in read mode:
        with open(STATS_FILE_PATH, 'r') as stats_file:
            stats_manager.load_from_csv(stats_file, base_timecode)
            
    start_time = base_timecode + 1.101
    print(start_time)
    video_manager.set_duration(start_time=start_time)

    # Set downscale factor to improve processing speed.
    video_manager.set_downscale_factor()

    # Start video_manager.
    video_manager.start()

    # Perform scene detection on video_manager.
    scene_manager.detect_scenes(frame_source=video_manager)

    # Obtain list of detected scenes.
    scene_list = scene_manager.get_scene_list(base_timecode)
    # Like FrameTimecodes, each scene in the scene_list can be sorted if the
    # list of scenes becomes unsorted.

    print('List of scenes obtained:')
    for i, scene in enumerate(scene_list):
        print('    Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
            i+1,
            scene[0].get_timecode(), scene[0].get_frames(),
            scene[1].get_timecode(), scene[1].get_frames(),))

    # We only write to the stats file if a save is required:
    if stats_manager.is_save_required():
        with open(STATS_FILE_PATH, 'w') as stats_file:
            stats_manager.save_to_csv(stats_file, base_timecode)

finally:
    video_manager.release()

In [ ]:
stats_df = pd.read_csv('video.stats.csv', header=1)
stats_df.iloc[120:140]

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf

init_notebook_mode(connected=True)
cf.set_config_file(world_readable=True, offline=True)

In [ ]:
data = [go.Histogram(x=stats_df['content_val'])]

iplot(data, filename='basic histogram')

In [ ]:
from secrets import randbelow
print(randbelow(10))

In [ ]:
scene_list